In [66]:
from bs4 import BeautifulSoup
import requests
import datetime
from datetime import date 
from datetime import datetime
import pandas as pd 
import re 
import numpy as np
import re
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
import sys
import warnings
from datetime import timedelta
import time as timp 
import dateutil.parser

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
#Get df of leagues 

def get_pinn_leagues(URL):
    chrome_path= r'C:\Users\Iris\Documents\Nico\NicoUni\Practicum\Python\Projects\WebDrivers\chromedriver.exe'
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(
        executable_path=chrome_path, options=chrome_options)
    
    browser.get(URL)
    timp.sleep(20)

    htmlSource = browser.page_source
    
    soup= BeautifulSoup(htmlSource, 'lxml')
    league_name= []
    links= []
    for element in soup.find_all('a',{'class':'style_noLeftIcon__3p7DJ style_compact__1hNoP style_supportFavoritesList__20Xov'}):
        links.append('https://www.pinnacle.com'+element['href'])
        league_name.append(element.text)
    
    link_frame= pd.DataFrame({
        'league': league_name,
        'link': links})
    
    p= re.compile('.*[^0-9]')
        
    for i in np.arange(0,len(link_frame)):
        link_frame['league'][i]= p.search(link_frame['league'][i]).group()
        
    return(link_frame)


#def 

In [197]:
def get_results(URL):
    import time as timp
    chrome_path= r'C:\Users\Iris\Documents\Nico\NicoUni\Practicum\Python\Projects\WebDrivers\chromedriver.exe'
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
    browser.get(URL)
    timp.sleep(10)
    soup= BeautifulSoup(browser.page_source, 'lxml')
    df= pd.DataFrame()
    error= []

    df= pd.DataFrame()
    
    for element in soup.find_all('div',{'class':'style_dateBar__2PS4O'}):
        count= len(element.text)
        if  count < 6:
            date= pd.to_datetime(datetime.now(), format="%d/%m/%Y")
        if 'Tomorrow' in element.text:
            days_add= timedelta(days= 1)
            date= pd.to_datetime(datetime.now(),format="%d/%m/%Y")+days_add
        else:
            try:
                temp_date= dateutil.parser.parse(element.text.split(',',1)[1][1:].replace(',',''))
                date= pd.to_datetime(temp_date, format= '%d/%m/%Y')
            except:
                date= element.text
        for tag in element.next_siblings:
            if tag['class'] == ['style_dateBar__2PS4O']:
                break
            if tag['class']== ['style_row__3_aBC','style_lastRow__3h8Pm'] or tag['class']== ['style_row__3_aBC']:
                try:
                    teams= tag.find_all('span', {'class':'style_participantName__CNiJz ellipsis'})
                    play_time= tag.find('span',{'class':'style_time__1_zpO ellipsis'})
                    odds= tag.find_all('span',{'class':'price'})
                    fest= pd.DataFrame({'date':date,
                                    'scrape_time':pd.to_datetime(datetime.now(), format="%d/%m/%Y %H:%M"),
                                    'time':play_time.text,
                         'home_team': teams[0].text,
                         'away_team': teams[1].text,
                         'PinnacleHome': odds[0].text,
                         'PinnacleDraw': odds[1].text,
                         'PinnacleAway': odds[2].text}, index= pd.Series(0))
                    df= df.append(fest)
                except:
                    continue
    return(df.reset_index(drop= True))

In [199]:
#get_results('https://www.pinnacle.com/en/soccer/egypt-premier-league/matchups')

Now I've got to 
1. Fix the dates !! Still Valid 
3. Make sure there is not a big time-lag--> Time lag must be a parameter to be optimised 
4. Prepare the data for the merge 
5. 

In [48]:
# Very Important script
# links= get_pinn_leagues('https://www.pinnacle.com/en/soccer/leagues')

results= pd.DataFrame()
errors= []

for i,element in enumerate(links['link'][0:1]):
    try:
        df= get_results(element)
        df['league']= links['league'][i]
        results= results.append(df)
        
    except:
        errors.append(element) 